In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

pdf_files = ["documents/pruebaLLM.pdf","documents/pruebaLLM2.pdf", "documents/MartinScorsese.pdf"]  # Agrega la lista de archivos PDF

all_documents = []

for document in pdf_files:
    loader = PyPDFLoader(document) # Carga el documento
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)    # Divide el texto en chunks
    documents = text_splitter.split_documents(data)
    all_documents.extend(documents)

print(f"Se cargaron y dividieron {len(all_documents)} fragmentos de texto.")

Se cargaron y dividieron 220 fragmentos de texto.


In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/home/martin/miniconda3/envs/rag/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_chroma import Chroma

# Crear un vector store de Chroma usando los documentos y el modelo de embeddings
vector_store = Chroma.from_documents(
    documents=all_documents,
    collection_name="pdfs_2",
    embedding=embeddings,  # Pasar el modelo de embeddings aquí
)

print(f"Se creó el vector store con {len(all_documents)} documentos.")

Se creó el vector store con 220 documentos.


In [4]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="pdfs_2",
    embedding_function=embeddings,
)

In [5]:
retriever = vector_store.as_retriever()

In [20]:
question = "Como es la filmografia de Martin Scorsese"
docs = vector_store.similarity_search(question, k=10)
len(docs)

10

In [21]:
docs[0]

Document(metadata={'page': 21, 'source': 'documents/MartinScorsese.pdf'}, page_content='500. Paris, 1996.\nMarino, Giuseppe Cario. Historiade laMafia. Barcelona: Ediciones B, 2002.\nMillas, Jorge. LaViolenciay sus Máscaras. Santiago: Aconcagua, 1976.\nRuiz, Raúl. PoéticaDel Cine. Santiago: Sudamericana, 2000.\nScorsese, Martínyotros. APersonalJourneyWithMartínScorseseThroughAmericanMovie.\nLondon: Akal, 1997.\nScorsese, Martínyotros. Conversaciones con Scorsese. Madrid: Plot, 1987.')

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM
ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

In [17]:
chain.invoke("pelicula y color favorito de Juan(pruebaLLM2.pdf)")

"La respuesta a la pregunta es:\n\n* Película favorita de Juan: 'Mulholland Drive'\n* Color favorito de Juan: Amarillo\n* Color que Juan odia: Verde"

In [10]:
chain.invoke("Quien es Martin Scorsese")

'Según el contexto proporcionado, Martin Scorsese es un cineasta conocido. También se mencionan varios libros y artículos relacionados con él, como "APersonalJourneyWithMartínScorseseThroughAmericanMovie" y "Conversaciones con Scorsese". Además, se hace referencia a películas de él, como "Mean Streets", que parece ser una película en la que también actúa.'

In [11]:
chain.invoke("Que relación tiene la violencia es sus películas?")

'La violencia es un tema central en las películas de Martin Scorsese, y parece que es una forma de expresión creativa y de exploración del ser humano. Según el texto, la violencia se presenta en diferentes formas en sus obras, como en "Mean Streets", donde se muestra como una parte inherente a la vida, y también en películas como "Casino" (1995), donde es un tema físico y psicológico.\n\nEn general, parece que Scorsese no aborda la violencia de manera didáctica o moralizante, sino que la presenta como una forma de expresión artística y de exploración del ser humano. La violencia también se relaciona con el sentido trágico de la vida y se presenta de manera sutil y elíptica.\n\nEn resumen, la violencia en las películas de Scorsese parece ser un tema complejo y multifacético que se relaciona con la expresión creativa, la exploración del ser humano y el sentido trágico de la vida.'

In [22]:
chain.invoke("Que hay en el documento sobre Goodfellas")

'En el documento, hay información sobre la película "Goodfellas" dirigida por Martin Scorsese. Se mencionan varias cosas:\n\n* La película es una evocación de la elegancia del gángster en la tradición del cine de género.\n* El protagonista, Charlie, convoca a un grupo que incluye a Michael y otros personajes.\n* La fiesta está determinada por ambigüedades perturbadoras.\n* Se menciona la relación entre Charlie y su padre, quien es un gángster y le enseña "enseñanzas de Paul Cicero".\n* También se habla sobre la autodestrucción de Henry Hill, el protagonista principal del film, y Ginger McKercher.\n* La película explora la moralidad gangsteril y cómo los personajes están atrapados en su mundo criminal.\n\nAdemás, se mencionan otros films de Martin Scorsese, como "Casino" y "Raging Bully", y cómo las estructuras narrativas de estas películas están relacionadas con "Goodfellas".'

In [24]:
chain.invoke("Resume el documento 'LA VIOLENCIA COMO OPCIÓN CREATIVA EN LA OBRA DE MARTÍN SCORSESE'")

'El documento "La violencia como opción creativa en la obra de Martin Scorsese" analiza la violencia como un elemento fundamental en la cinematografía de Martin Scorsese. El autor argumenta que la violencia es una forma de expresión creativa y poética, que se puede encontrar en muchos de sus filmes.\n\nSegún el documento, Scorsese define la violencia como "esencial" en su obra y lo utiliza para explorar temas morales y filosóficos. El autor menciona específicamente cómo Scorsese aborda la violencia en películas como "Mean Streets" y "Taxi Driver", destacando la importancia de las relaciones con otros autores, como Fellini.\n\nEl documento también destaca la presencia de un fatalismo siciliano en la obra de Scorsese, que se refleja en su uso de la violencia para expresar un sentido de desesperanza y moralidad. Sin embargo, el autor sostiene que Scorsese ofrece una consecuencia moral a la violencia, explorándola de manera poética y creativa.\n\nEn general, el documento presenta una visió

In [25]:
chain.invoke("Que relación tiene la mafia en sus películas?")

'La mafia parece ser un tema central en las películas de Scorsese, como se desprende del contexto proporcionado. La mafia es descrita como una familia perversa y agrega que "No hay escapatoria" para un miembro de la familia Corleone. También se menciona que la mafia está dentro del Estado y que su directora convirtió en un modo de vida.\n\nAdemás, se destaca la importancia de la mafia en sus películas al ser descrita como una organización fuerte y poderosa, con prioridades en la familia y la iglesia. También se menciona que la mafia es capaz de ser a la vez manipuladora y brutal, lo que sugiere una complejidad en su naturaleza.\n\nEn resumen, la relación de la mafia en las películas de Scorsese parece ser fundamental, siendo descrita como una parte integral del tejido social y una forma de vida para muchos personajes.'